In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment/Dataset/sample_submission.csv
/kaggle/input/shl-intern-hiring-assessment/Dataset/train.csv
/kaggle/input/shl-intern-hiring-assessment/Dataset/test.csv
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_885.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_1142.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_1006.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_817.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_765.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_508.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_257.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_330.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_328.wav
/kaggle/input/shl-intern-hiring-ass

In [2]:
#load the training dataset
train_df=pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/Dataset/train.csv")
train_df.head()

,filename,label
0,audio_710.wav,1.0
1,audio_1265.wav,1.0
2,audio_1114.wav,1.5
3,audio_946.wav,1.5
4,audio_1127.wav,2.0


In [3]:
train_df.label.value_counts()

label
5.0    150
4.0     82
4.5     68
3.0     51
2.0     44
2.5     26
3.5     19
1.0      2
1.5      2
Name: count, dtype: int64

In [4]:
#extract features from train dataset using mlfccs
import librosa
def extract_features(filename):
    data, sample_rate=librosa.load(filename)
    mfccs_features=librosa.feature.mfcc(y=data,sr=sample_rate, n_mfcc=30)
    mfccs_scaled=np.mean(mfccs_features.T, axis=0)
    zcr = np.mean(librosa.feature.zero_crossing_rate(data))
    chroma = np.mean(librosa.feature.chroma_stft(y=data, sr=sample_rate))
    contrast = np.mean(librosa.feature.spectral_contrast(y=data, sr=sample_rate))
    full_feature = np.hstack([mfccs_scaled, zcr, chroma, contrast])

    return full_feature
    

In [5]:
import numpy as np
from tqdm import tqdm

train_path="/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/train"

extracted=[]
for index_num,row in tqdm(train_df.iterrows()):
    fn=os.path.join(os.path.abspath(train_path), str(row['filename']))
    data=extract_features(fn)
    final_label=row['label']
    extracted.append([data, final_label])

444it [05:22,  1.38it/s]


In [6]:
df=pd.DataFrame(extracted, columns=['feature','label'])

In [7]:
X=np.array(df.feature.tolist())
X.shape

(444, 33)

In [8]:
y=np.array(df.label.tolist())
y.shape

(444,)

In [9]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

2025-05-03 08:49:55.997946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746262196.263823      12 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746262196.345863      12 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [11]:
X_train.shape, X_test.shape

((355, 33), (89, 33))

In [12]:
y_train.shape, y_test.shape

((355,), (89,))

In [13]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error

# Define parameter distributions
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 3, 5]
}

# Use MAE as the scoring metric
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,  # Increase for deeper search
    scoring=mae_scorer,
    cv=3,
    verbose=2,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

# Best results
print("Best MAE:", -random_search.best_score_)
print("Best Parameters:", random_search.best_params_)


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best MAE: 0.7038715706304884
Best Parameters: {'subsample': 0.8, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 1.0}


In [14]:
xgbmodel=XGBRegressor(subsample=0.8,n_estimators=500, min_child_weight=3,
                     max_depth=5, learning_rate=0.05,gamma=0.1,colsample_bytree=1)

xgbmodel.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.1, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=3, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [15]:
from sklearn.metrics import mean_squared_error, accuracy_score,mean_absolute_error
y_pred=xgbmodel.predict(X_test)
print("mean_squared_error:", mean_squared_error(y_test,y_pred))
print("mean_abs_error:", mean_absolute_error(y_test,y_pred))


mean_squared_error: 0.7967121654792886
mean_abs_error: 0.7378328676973835


In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error

# Define parameter distributions
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# MAE scorer
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Initialize model
rf = RandomForestRegressor(random_state=42)

# RandomizedSearchCV
random_search_rf = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=30,
    scoring=mae_scorer,
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Fit search
random_search_rf.fit(X_train, y_train)

# Output best results
print("Best MAE (RF):", -random_search_rf.best_score_)
print("Best Parameters (RF):", random_search_rf.best_params_)


Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

Best MAE (RF): 0.7131846573375112
Best Parameters (RF): {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 40, 'bootstrap': False}


In [17]:

randommodel=RandomForestRegressor(n_estimators=500,min_samples_split=2, min_samples_leaf=1, 
                                 max_depth=40,bootstrap=False, max_features='log2')
randommodel.fit(X_train,y_train)
y_pred=randommodel.predict(X_test)
print(mean_absolute_error(y_test,y_pred))
mean_squared_error(y_test,y_pred)

0.7481460674157303


0.8210522584269662

In [18]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error

# Define hyperparameter space
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [20, 31, 40, 50],
    'max_depth': [-1, 5, 10, 20],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# MAE scorer
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Initialize model
lgbm = LGBMRegressor(random_state=42)

# Randomized search
random_search_lgbm = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=param_dist,
    n_iter=30,
    scoring=mae_scorer,
    cv=3,
    n_jobs=-1
)

random_search_lgbm.fit(X_train, y_train)

# Best results
print("Best MAE (LightGBM):", -random_search_lgbm.best_score_)
print("Best Parameters (LightGBM):", random_search_lgbm.best_params_)

[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.2, max_depth=5, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.3s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.1, max_depth=7, min_child_weight=3, n_estimators=500, subsample=0.8; total time=   0.5s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=200, subsample=0.8; total time=   0.9s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=500, subsample=0.8; total time=   0.6s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=500, subsample=0.8; total time=   0.6s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_depth=5, min_child_weight=3, n_estimators=200, subsample=1.0; total time=   1.0s
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.05, max_depth=5, min_child_weight=3, n_estimators=200, subsample=0.

In [19]:
lgbm = LGBMRegressor(random_state=42,subsample=0.6, num_leaves=40,n_estimators=100,
                    max_depth=20, learning_rate=0.05,colsample_bytree=0.8)
lgbm.fit(X_train,y_train)
lg_pred=lgbm.predict(X_test)

print(mean_squared_error(y_test,lg_pred))
print(mean_absolute_error(y_test,lg_pred))




[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000179 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3934
[LightGBM] [Info] Number of data points in the train set: 355, number of used features: 33
[LightGBM] [Info] Start training from score 4.001408
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [20]:
from sklearn.linear_model import Ridge

model_ridge = Ridge()


model_ridge.fit(X_train,y_train)
# Predict on test set
pred_rf = randommodel.predict(X_test)
pred_xgb = xgbmodel.predict(X_test)
pred_ridge = model_ridge.predict(X_test)


# Simple average ensemble
ensemble_pred = (pred_rf + pred_xgb + pred_ridge+lg_pred) / 4
p=np.round(ensemble_pred*2)/2

# Evaluate
mae = mean_absolute_error(y_test, ensemble_pred)
print(f"Ensemble MAE: {mae}")
print("mse",mean_squared_error(y_test,p))

Ensemble MAE: 0.7445706282566912
mse 0.9241573033707865


In [21]:
import numpy as np
from tqdm import tqdm

test_path="/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test"
test_df=pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/Dataset/test.csv")

extracted=[]
for index_num,row in tqdm(test_df.iterrows()):
    fn=os.path.join(os.path.abspath(test_path), str(row['filename']))
    data=extract_features(fn)
    
    extracted.append([data])

204it [02:18,  1.47it/s]


In [22]:
df2=pd.DataFrame(extracted, columns=['feature'])
test_X=np.array(df2.feature.tolist())

random=randommodel.predict(test_X)
xg=xgbmodel.predict(test_X)
lg=lgbm.predict(test_X)
rg=model_ridge.predict(test_X)

final_preds=(random+xg+lg+rg)/4

print(final_preds)

[3.41108217 4.05740781 3.37888736 3.13161529 3.24304977 3.68741188
 4.27920673 3.78972747 2.9895971  3.33246093 3.05291975 3.06599708
 3.901705   2.94896033 3.90009598 3.21895109 3.82286424 3.27243266
 3.04808674 3.07126283 3.69044048 3.75002679 3.7581977  2.69799612
 3.90628934 4.07942018 3.51867821 3.44420696 4.30361128 3.92081845
 4.17009856 3.21034285 4.22401076 3.45702124 4.1998789  3.79146115
 3.21173456 3.78687605 4.16941889 3.60827772 4.32436056 3.58213531
 3.61643861 3.47209192 3.30567706 3.69726529 3.82350337 3.95923141
 3.92137615 4.28220566 4.45156057 4.08826654 3.52584681 4.20478375
 3.09108933 4.38451085 3.71575101 4.24298698 3.21600022 3.77576718
 3.95061202 3.57760932 3.60527296 4.17387129 4.46056517 4.44625209
 4.09689397 3.97347355 3.55516048 4.51955541 2.97654767 3.52264844
 3.77149113 3.49169201 3.13832516 4.18826197 3.72662943 3.3596048
 3.38380556 4.58001475 4.04492619 4.30357796 3.41938096 4.03625121
 3.88387666 3.49569932 3.73032046 4.68878369 3.91338063 4.24608

In [23]:
fp=np.round(final_preds*2)/2

In [24]:
fp

array([3.5, 4. , 3.5, 3. , 3. , 3.5, 4.5, 4. , 3. , 3.5, 3. , 3. , 4. ,
       3. , 4. , 3. , 4. , 3.5, 3. , 3. , 3.5, 4. , 4. , 2.5, 4. , 4. ,
       3.5, 3.5, 4.5, 4. , 4. , 3. , 4. , 3.5, 4. , 4. , 3. , 4. , 4. ,
       3.5, 4.5, 3.5, 3.5, 3.5, 3.5, 3.5, 4. , 4. , 4. , 4.5, 4.5, 4. ,
       3.5, 4. , 3. , 4.5, 3.5, 4. , 3. , 4. , 4. , 3.5, 3.5, 4. , 4.5,
       4.5, 4. , 4. , 3.5, 4.5, 3. , 3.5, 4. , 3.5, 3. , 4. , 3.5, 3.5,
       3.5, 4.5, 4. , 4.5, 3.5, 4. , 4. , 3.5, 3.5, 4.5, 4. , 4. , 3.5,
       3.5, 3. , 4. , 4. , 3.5, 3. , 3.5, 4. , 4. , 3.5, 3. , 3.5, 4. ,
       3.5, 4. , 4.5, 3. , 4. , 4. , 3. , 3.5, 3.5, 4.5, 4.5, 3.5, 4. ,
       3.5, 4. , 4.5, 4.5, 3.5, 3. , 3. , 3.5, 4.5, 4. , 3.5, 4.5, 4.5,
       3. , 4. , 4.5, 4. , 4. , 4. , 4.5, 4.5, 4. , 4.5, 4. , 3.5, 4. ,
       4. , 3.5, 4. , 4.5, 4. , 4. , 4. , 4.5, 4. , 4.5, 3.5, 4.5, 4.5,
       4.5, 4. , 4. , 4.5, 4. , 3.5, 4. , 4. , 5. , 4. , 4.5, 4. , 4. ,
       4.5, 4. , 4. , 4. , 4. , 4.5, 4. , 4. , 4.5, 4. , 4.5, 4.

In [25]:
submission=pd.DataFrame({'filename':test_df.filename,'label':final_preds})


In [26]:
submission.to_csv('submission.csv',index=False)